# Perform inference

## 1. Description

Here, I use the model to perform query questions.

Each question will assess how different compartments of the predation process influence prey capture.

**Question 1 :** How does expertise influence foraging decisions and prey capture?

**Question 2 :** How does past experience and memory influence foraging decisions and prey capture?

**Question 3 :** How does the current interaction influence foraging decision and prey capture?

## 2. Prepare the session

Load the libraries

In [18]:
# To manipulate data
import pandas as pd

# To perform inference
from pgmpy.inference import ApproxInference

# To import the data
from joblib import load
import os

Import the model object.

In [10]:
# Define the parent directory
parent_dir = os.path.abspath(
    os.path.join(os.getcwd(), os.pardir)
)

# Define the subfolder directory
output_folder = os.path.join(
    parent_dir, "outputs"
)

# Name of the model output file
file_name = "BayesNet-fit.joblib"

model = load(
    os.path.join(
        output_folder,
        file_name
    )
)

## 3. Approximate inference with sampling

I start by creating an inference object called infer

In [ ]:
infer = ApproxInference(model)

### 3.1 Question 1 - Expertise

Here, I perform a query asking the question of whether the probability of capture given the foraging mode changes when predators are advanced.

In [62]:
q1 = infer.query(
    variables = ["prey_captured"],
    evidence = {"xp_level": "advanced", "pred_speed_bin": "Faster"},
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

In [63]:
print(q1)

+------------------+----------------------+
| prey_captured    |   phi(prey_captured) |
+==================+======================+
| prey_captured(0) |               0.3252 |
+------------------+----------------------+
| prey_captured(1) |               0.6748 |
+------------------+----------------------+


How about when predators are novice?

In [66]:
q2 = infer.query(
    variables = ["prey_captured"],
    evidence = {"xp_level": "advanced", "pred_speed_bin": "Average"},
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

In [67]:
print(q2)

+------------------+----------------------+
| prey_captured    |   phi(prey_captured) |
+==================+======================+
| prey_captured(1) |               0.8064 |
+------------------+----------------------+
| prey_captured(0) |               0.1936 |
+------------------+----------------------+


In [36]:
q3 = infer.query(
    variables = ["prey_captured", "chases_persec_bin"],
    #evidence = {"prey_avg_speed_bin" : "Faster"},
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

In [37]:
print(q3)

+------------------+----------------------------+----------------------------------------+
| prey_captured    | chases_persec_bin          |   phi(prey_captured,chases_persec_bin) |
+==================+============================+========================================+
| prey_captured(1) | chases_persec_bin(Low)     |                                 0.2910 |
+------------------+----------------------------+----------------------------------------+
| prey_captured(1) | chases_persec_bin(Average) |                                 0.4018 |
+------------------+----------------------------+----------------------------------------+
| prey_captured(1) | chases_persec_bin(High)    |                                 0.0772 |
+------------------+----------------------------+----------------------------------------+
| prey_captured(0) | chases_persec_bin(Low)     |                                 0.0658 |
+------------------+----------------------------+----------------------------------------+

In [40]:
q4 = infer.query(
    variables = ["prey_captured", "prey_avg_speed_bin", "xp_level"],
    #evidence = {"prey_avg_speed_bin" : "Faster"},
    joint = True,
    n_samples = 5000,
    seed = 123,
    show_progress = False
)

In [41]:
print(q4)

+------------------+-----------------------------+--------------------+--------------------------------------------------+
| prey_captured    | prey_avg_speed_bin          | xp_level           |   phi(prey_captured,prey_avg_speed_bin,xp_level) |
+==================+=============================+====================+==================================================+
| prey_captured(1) | prey_avg_speed_bin(Average) | xp_level(advanced) |                                           0.0758 |
+------------------+-----------------------------+--------------------+--------------------------------------------------+
| prey_captured(1) | prey_avg_speed_bin(Average) | xp_level(interm)   |                                           0.1208 |
+------------------+-----------------------------+--------------------+--------------------------------------------------+
| prey_captured(1) | prey_avg_speed_bin(Average) | xp_level(novice)   |                                           0.0596 |
+---------------